In [1]:
import numpy as np
import pandas as pd


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.models import Model
from keras.layers import LSTM

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from math import sqrt

Using TensorFlow backend.


# Dane

In [24]:
data = pd.read_csv('dane-pre.csv')

data.head(14)

,timestamp,MpKrakAlKras-PM10-1g,Weather,Temp(C),Wind(m/s),Direction(deg),Humidity,Barometer(mbar),PM10_Forecast_3h,PM10_Forecast_8h,PM10_Forecast_12h
0,2017-01-01 01:00:00,213.733,Ice fog.,-6.0,1.0,0.0,0.93,10.24,244.690,187.320,121.805
1,2017-01-01 02:00:00,213.868,Ice fog.,-7.0,2.0,190.0,0.93,10.24,224.673,181.194,119.096
2,2017-01-01 03:00:00,221.923,Ice fog.,-4.0,3.0,200.0,0.80,10.23,212.159,161.198,121.681
3,2017-01-01 04:00:00,244.690,Ice fog.,-7.0,3.0,210.0,0.86,10.22,198.375,136.218,138.353
4,2017-01-01 05:00:00,224.673,Ice fog.,-6.0,3.0,200.0,0.80,10.22,191.881,121.805,150.415
5,2017-01-01 06:00:00,212.159,Ice fog.,-7.0,2.0,190.0,0.86,10.22,187.320,119.096,161.587
6,2017-01-01 07:00:00,198.375,Ice fog.,-8.0,3.0,210.0,0.86,10.21,181.194,121.681,197.133
7,2017-01-01 08:00:00,191.881,Ice fog.,-8.0,1.0,0.0,0.93,10.21,161.198,138.353,217.923
8,2017-01-01 09:00:00,187.320,Ice fog.,-6.0,2.0,200.0,0.86,10.21,136.218,150.415,250.248
9,2017-01-01 10:00:00,181.194,Ice fog.,-3.0,2.0,200.0,0.74,10.21,121.805,161.587,227.516


In [25]:
X = data[["MpKrakAlKras-PM10-1g","Direction(deg)","Temp(C)","Wind(m/s)","Humidity","Barometer(mbar)"]]

y = data[["PM10_Forecast_3h","PM10_Forecast_8h","PM10_Forecast_12h"]]

In [ ]:
data.head(5)

In [26]:
scaler = StandardScaler()
X_scale = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X_scale

,MpKrakAlKras-PM10-1g,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar)
0,3.406057,-1.229537,-1.641839,-1.06963,0.922469,1.009585
1,3.408971,0.461359,-1.747499,-0.62705,0.922469,1.009585
2,3.582857,0.550354,-1.430521,-0.18447,0.165148,0.881591
3,4.074335,0.639348,-1.747499,-0.18447,0.514681,0.753597
4,3.642222,0.550354,-1.641839,-0.18447,0.165148,0.753597
...,...,...,...,...,...,...
17298,-1.043013,1.173316,-0.796564,0.25811,0.922469,1.009585
17299,-1.012067,1.351305,-0.796564,0.25811,0.572936,1.137579
17300,-0.966574,1.440300,-0.690905,0.70069,0.223403,1.265572
17301,-0.969935,1.440300,-0.690905,1.14327,0.223403,1.265572


In [27]:
#weather_id trzeba by dodać jako one-hot a nie słownik, tylko że wtedy mamy 100 dodatkowych kolumn

X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.1,
                                                    random_state=0)

In [28]:
train_X = X_train.values
test_X = X_test.values 
train_y = y_train.values
test_y = y_test.values


In [29]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

# Model

In [30]:
model = Sequential()
# 50 neurons in first hidden layer
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(3,kernel_initializer='normal', activation='linear'))
model.compile(loss='mae', optimizer='adam')

In [32]:
model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=0, shuffle=False)

In [22]:
y_pred = model.predict(test_X)
rms = sqrt(mean_squared_error(y_test, y_pred))
rms

32.751356502983064

In [38]:
now = np.array([[20,210,-8,3,0.8 ,10]])
n=scaler.transform(now)
nnn = n.reshape((n.shape[0], 1, n.shape[1]))

pred = model.predict(nnn)
pred
#def predict(condtions):
#    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=96)
#    score = model.predict([x_test])[0]

array([[34.857307, 34.572086, 34.939114]], dtype=float32)

array([[ 36.7729 ,  20.606  ,  24.0241 ],
       [132.06   , 169.328  ,  96.7489 ],
       [105.845  ,  48.0869 , 124.753  ],
       ...,
       [ 15.4147 ,   8.61533,  16.6127 ],
       [ 22.3883 ,  32.3156 ,  46.3553 ],
       [ 43.7125 ,  93.7422 , 106.905  ]])